In [ ]:
# To Z-score the weighted degree centrality maps

In [1]:
import os
import glob

import numpy as np
from sklearn.preprocessing import StandardScaler

import nibabel as nib
from nilearn.maskers import NiftiMasker
from nilearn.image import concat_imgs
from nilearn.image import math_img
from nilearn import plotting

In [2]:
# Load an image from a single brain:
def img_load(folder, img):
    img_path = os.path.join(folder, img)
    fmri_image = nib.load(img_path)
    #print(fmri_image.shape)
    return fmri_image

In [3]:
# Extract the timeseries from a single session
def extract_time_series(fmri_image, mask):
    brain_masker = NiftiMasker(mask, memory='nilearn_cache', memory_level=1, verbose=0)
    brain_time_series = brain_masker.fit_transform(fmri_image)
    return brain_time_series, brain_masker

In [4]:
# Z-score
def z_score_dc(dc_array):
    dc = dc_array.reshape(-1, 1)
    scaler = StandardScaler()
    dc_scaled = scaler.fit_transform(dc)
    return dc_scaled

In [5]:
# Transform the FC or DC array back into a NIfTI volume and save
def array_to_nifti(brain_masker,array):
    img = brain_masker.inverse_transform(array.T)
    return img

In [6]:
def z_score_multi(rootDIR, SUBlist):
    for i in range(len(SUBlist)):
        
        # load an image from a single brain:
        dc_image = img_load(rootDIR, SUBlist[i])
        sub_id = SUBlist[i][73:79]
        print("Participant " + sub_id)
        #print("The image dimensions are: " + str(dc_image.shape))
        
        # create a mask from the participant image (otherwise, there will be regions without signal because of the 
        # discrepancy between whole-brain GM and participant-specific mask
        mask = math_img('img > 0', img=dc_image)
        print("Mask created.")
        
        # create a NiftiMasker object and extract the time series
        dc_vals, brain_masker = extract_time_series(dc_image, mask)
        print("DC extracted.")
        
        # z-score the DC matrix
        z_scored_dc_vals = z_score_dc(dc_vals)
        print("DC matrix z-scored.")
                
        # convert to NIfTI and save
        z_scored_image = array_to_nifti(brain_masker, z_scored_dc_vals)
        image_name = sub_id + "_wDC_Z"
        nib.save(z_scored_image, image_name)
        print("Saved!")
    
    
    print("Done!")

In [7]:
work_dir = '/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Static_FC/DC_data'

In [8]:
sub_list = sorted(glob.glob("/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Static_FC/DC_data/MAGACAN????_wDC_raw.nii"))

In [ ]:
z_score_multi(work_dir, sub_list)